In [1]:
import cv2
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import math
from pandas import Series, DataFrame

import os ,time

import glob
import random
from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torch.optim.lr_scheduler import _LRScheduler
import torch_optimizer as optim

import numpy as np
import pandas as pd, cv2
from tqdm import tqdm
from easydict import EasyDict
from sklearn.preprocessing import LabelEncoder #Label Encoder로 숫자로 변경함
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from ptflops import get_model_complexity_info

from collections import Counter
import argparse
import neptune.new as neptune
import timm

#안하면 오류남
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore')

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
pd.set_option('display.max_row',10) #가로 제한 수 10 밑에서 풀어버려서;
train_data = pd.read_csv("../Data/train.csv")
display(train_data)

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good
...,...,...,...,...,...
2995,2995,12995.png,screw,good,screw-good
2996,2996,12996.png,wood,good,wood-good
2997,2997,12997.png,zipper,good,zipper-good
2998,2998,12998.png,tile,oil,tile-oil


In [3]:
class_uni = train_data["class"].unique()
class_uni

array(['transistor', 'capsule', 'wood', 'bottle', 'screw', 'cable',
       'carpet', 'hazelnut', 'pill', 'metal_nut', 'zipper', 'leather',
       'toothbrush', 'tile', 'grid'], dtype=object)

In [4]:
df_load = []
for i in class_uni:
    df_load.append(pd.read_csv(f"../Data/{i}_df.csv"))

In [5]:
display(df_load[0])

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,2,10002.png,transistor,good,transistor-good
2,9,10009.png,transistor,good,transistor-good
3,42,10042.png,transistor,good,transistor-good
4,49,10049.png,transistor,good,transistor-good
...,...,...,...,...,...
161,2904,12904.png,transistor,cut_lead,transistor-cut_lead
162,2951,12951.png,transistor,good,transistor-good
163,2972,12972.png,transistor,good,transistor-good
164,2976,12976.png,transistor,good,transistor-good


In [6]:
aug = iaa.Sequential([
    iaa.Rotate((-45,45)),
    iaa.Fliplr(0.5),
])

In [44]:
def augData(df, transform,times,save_path):
    
    os.makedirs(save_path, exist_ok=True)
    file_name = df['file_name'].values
    labels = df['label'].values
    class_name = df['class'][0]
    transform = transform
    data_path = '../Data/train data/'
    save_path = save_path
    states = df['state'].values
    augDatas = []
    times = times
    
    print(f"데이터셋 로드 성공 데이터 {class_name} Size : {len(file_name)}")
    for index in range(len(file_name)):
        img_path = os.path.join(data_path, file_name[index])
        image = cv2.imread(img_path)
        label = labels[index]
        state = states[index]
        if state != "good":
            print(f"데이터 증강,{label},{state}")
            for i in range(times):
                aug_image = aug(images = [image])
                aug_image = np.squeeze(aug_image, axis=0)
                
                save_aug = os.path.join(save_path,f"{i}{str(class_name)}{file_name[index]}")
                
                vec = [f"{i}{str(class_name)}{file_name[index]}",class_name,state,label]
                cv2.imwrite(save_aug,aug_image)
                augDatas.append(vec)
                
                if i == times - 1:
                    save_aug = os.path.join(save_path,f"{class_name}{file_name[index]}")
                    vec = [f"{class_name}{file_name[index]}",class_name,state,label]
                    cv2.imwrite(save_aug,image)
                    augDatas.append(vec)
        else :
            save_aug = os.path.join(save_path,f"{class_name}{file_name[index]}")
            vec = [f"{class_name}{file_name[index]}",class_name,state,label]
            cv2.imwrite(save_aug,image)
            augDatas.append(vec)

    df = DataFrame(augDatas, columns = ["file_name","class","state","label"])
    df.to_csv(f'../Data/aug/{class_name}_df.csv', index=False)
    print(f"증강 한후의 갯수 :{len(df)}")

In [45]:
for idx, name in enumerate(class_uni):
    augData(df_load[idx],aug,5,f"../Data/aug/{name}/")

데이터셋 로드 성공 데이터 transistor Size : 166
데이터 증강,transistor-bent_lead,bent_lead
데이터 증강,transistor-damaged_case,damaged_case
데이터 증강,transistor-misplaced,misplaced
데이터 증강,transistor-misplaced,misplaced
데이터 증강,transistor-cut_lead,cut_lead
데이터 증강,transistor-damaged_case,damaged_case
데이터 증강,transistor-damaged_case,damaged_case
데이터 증강,transistor-damaged_case,damaged_case
데이터 증강,transistor-bent_lead,bent_lead
데이터 증강,transistor-bent_lead,bent_lead
데이터 증강,transistor-damaged_case,damaged_case
데이터 증강,transistor-misplaced,misplaced
데이터 증강,transistor-cut_lead,cut_lead
증강 한후의 갯수 :231
데이터셋 로드 성공 데이터 capsule Size : 196
데이터 증강,capsule-squeeze,squeeze
데이터 증강,capsule-squeeze,squeeze
데이터 증강,capsule-crack,crack
데이터 증강,capsule-poke,poke
데이터 증강,capsule-crack,crack
데이터 증강,capsule-squeeze,squeeze
데이터 증강,capsule-crack,crack
데이터 증강,capsule-faulty_imprint,faulty_imprint
데이터 증강,capsule-scratch,scratch
데이터 증강,capsule-crack,crack
데이터 증강,capsule-poke,poke
데이터 증강,capsule-crack,crack
데이터 증강,capsule-poke,poke
데이터 증강,capsule-

데이터 증강,hazelnut-print,print
데이터 증강,hazelnut-hole,hole
데이터 증강,hazelnut-print,print
데이터 증강,hazelnut-print,print
데이터 증강,hazelnut-cut,cut
데이터 증강,hazelnut-print,print
데이터 증강,hazelnut-cut,cut
데이터 증강,hazelnut-crack,crack
데이터 증강,hazelnut-hole,hole
데이터 증강,hazelnut-hole,hole
데이터 증강,hazelnut-cut,cut
데이터 증강,hazelnut-cut,cut
데이터 증강,hazelnut-hole,hole
데이터 증강,hazelnut-hole,hole
데이터 증강,hazelnut-crack,crack
데이터 증강,hazelnut-cut,cut
데이터 증강,hazelnut-crack,crack
데이터 증강,hazelnut-print,print
데이터 증강,hazelnut-print,print
데이터 증강,hazelnut-cut,cut
데이터 증강,hazelnut-crack,crack
데이터 증강,hazelnut-cut,cut
증강 한후의 갯수 :430
데이터셋 로드 성공 데이터 pill Size : 240
데이터 증강,pill-pill_type,pill_type
데이터 증강,pill-scratch,scratch
데이터 증강,pill-crack,crack
데이터 증강,pill-color,color
데이터 증강,pill-color,color
데이터 증강,pill-contamination,contamination
데이터 증강,pill-faulty_imprint,faulty_imprint
데이터 증강,pill-crack,crack
데이터 증강,pill-pill_type,pill_type
데이터 증강,pill-combined,combined
데이터 증강,pill-faulty_imprint,faulty_imprint
데이터 증강,pill-scratch,scratch
데이터 증강

In [ ]:
mage = cv2.imread("/home/twogudack/Czion_anomaly/Data/train data/10000.png")

In [ ]:
cv2.imshow("test",mage)
cv2.waitKey(0)
cv2.destroyAllWindows